In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
lgb_sub = pd.read_csv('prob/sub_lgb.csv')
xgb_sub = pd.read_csv('prob/sub_xgb.csv')
cbt_sub = pd.read_csv('prob/sub_cbt.csv')

In [3]:
lgb_oof = pd.read_csv('prob/oof_lgb.csv')
xgb_oof = pd.read_csv('prob/oof_xgb.csv')
cbt_oof = pd.read_csv('prob/oof_cbt.csv')

In [4]:
sub = lgb_sub.copy()
sub.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
sub['prob_xgb'] = xgb_sub['prob'].values.tolist()
sub['prob_cbt'] = cbt_sub['prob'].values.tolist()
sub.head()

,id,prob_lgb,prob_xgb,prob_cbt
0,800000,0.189512,0.052258,0.054796
1,800001,0.635726,0.308782,0.331217
2,800002,0.577289,0.264748,0.339922
3,800003,0.626123,0.299554,0.298638
4,800004,0.712837,0.363835,0.388608


In [5]:
oof = lgb_oof.copy()
oof.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
oof['prob_xgb'] = xgb_oof['prob'].values.tolist()
oof['prob_cbt'] = cbt_oof['prob'].values.tolist()
oof.head()

,id,prob_lgb,isDefault,prob_xgb,prob_cbt
0,1,0.534962,0.0,0.298526,0.240333
1,23,0.674715,1.0,0.341080,0.344511
2,28,0.341769,0.0,0.115245,0.120330
3,32,0.639611,0.0,0.278753,0.311702
4,42,0.625265,1.0,0.295865,0.295955


In [6]:
auc1 = roc_auc_score(oof['isDefault'], oof['prob_lgb'])
auc2 = roc_auc_score(oof['isDefault'], oof['prob_xgb'])
auc3 = roc_auc_score(oof['isDefault'], oof['prob_cbt'])

auc1, auc2, auc3

(0.742672420904967, 0.743330876642909, 0.7434870090279698)

In [7]:
w_lgb = 0.5
w_xgb = 0.5
w_cbt = 0.5

oof['prob'] = oof['prob_lgb'] ** w_lgb * oof['prob_xgb'] ** w_xgb * oof['prob_cbt'] ** w_cbt
auc = roc_auc_score(oof['isDefault'], oof['prob'])
auc

0.744289702409715

In [8]:
sub['prob'] = sub['prob_lgb'] ** w_lgb * sub['prob_xgb'] ** w_xgb * sub['prob_cbt'] ** w_cbt
sub.head()

,id,prob_lgb,prob_xgb,prob_cbt,prob
0,800000,0.189512,0.052258,0.054796,0.023295
1,800001,0.635726,0.308782,0.331217,0.254987
2,800002,0.577289,0.264748,0.339922,0.227931
3,800003,0.626123,0.299554,0.298638,0.236668
4,800004,0.712837,0.363835,0.388608,0.317470


In [9]:
sub.rename({'prob': 'isDefault'}, axis=1, inplace=True)
sub[['id', 'isDefault']].to_csv('sub/stacking.csv', index=False)